In [1]:
import numpy as np 

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn import decomposition
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import NuSVR # nu-SVR, implement from libsvm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import TSNE
from sklearn import preprocessing
# different regressor test

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPClassifier

import pandas as pd # process txt
from scipy.io import loadmat
from PIL import Image

from collections import defaultdict

# 645x312的输入，312x1024的矩阵，变换为645x1024的输出
# 43 * 15
import torch 
#import torch.nn as nn
#import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [2]:
folder = "G:/dataset/sun/"

standard_path = "G:/dataset/standard_split/SUN/"
proposed_path = "G:/dataset/proposed_split/SUN/"


cls_to_idx = {}
cnt = 0 
with open(standard_path + "allclasses.txt", "r", encoding='utf-8') as f: # TBD
     for row in f.readlines():
         row = row.rstrip()
         cls_to_idx[row] = cnt
         cnt = cnt + 1

sstrain, sstest = [], []
pstrain, pstest = [], []

# Standard Split 
with open(standard_path + "trainvalclasses.txt", "r", encoding='utf-8') as f:
    for row in f.readlines():
        row = row.rstrip()
        sstrain.append(cls_to_idx[row])

with open(standard_path + "testclasses.txt", "r", encoding='utf-8') as f:
    for row in f.readlines():
        row = row.rstrip()
        sstest.append(cls_to_idx[row])

print("standard_split:", len(sstrain), len(sstest))
# transform List(str) -> List(int)

# Proposed Split
with open(proposed_path + "trainvalclasses.txt", "r", encoding='utf-8') as f:
    for row in f.readlines():
        row = row.rstrip()
        pstrain.append(cls_to_idx[row])

with open(proposed_path + "testclasses.txt", "r", encoding='utf-8') as f:
    for row in f.readlines():
        row = row.rstrip()
        pstest.append(cls_to_idx[row])

print("proposed_split:", len(pstrain), len(pstest))

# Random Train & Test Class Split

train_class, test_class = [], [] 
X_class = list(range(717)) # label start from 0, you can adjust it
train_class, test_class = train_test_split(X_class, test_size=0.2)

# len(train_class), len(test_class) == (573, 144)

train_class, test_class = sstrain, sstest
train_class, test_class = pstrain, pstest



standard_split: 645 72
proposed_split: 645 72


In [3]:
# semantic embedding of SUN, 717 scenes, 102 attributes
# fork s.e. from essay "evaluation"
#se = folder + "sun_se_continuous.txt"

#semat = att['att'].T # transpose
#semat = np.loadtxt(se, delimiter=" ", encoding='utf-8')


class sunRead:
    def __init__(self, p, train_split):
        """
        p: 数据集存放路径
        train_split: 给出训练集
        """
        X_class = list(range(717))
        train_class = train_split
        test_class = list(filter(lambda i: i not in train_class, X_class))
        self.path = p
        # labels
        yp = self.path + "custom-sun-labels.txt"
        y = np.loadtxt(yp, delimiter=" ", encoding='utf-8')
        # visual features 2048 dimensions
        xp = self.path + "sun-features.txt"
        x = np.loadtxt(xp, delimiter=" ", encoding='utf-8')
        i1 = np.isin(y, train_class)
        i2 = np.isin(y, test_class)
        self.X_train, self.X_test = x[i1], x[i2]
        self.y_train, self.y_test = y[i1], y[i2]
    def train_data(self):
        return self.X_train, self.y_train
    def test_data(self):
        return self.X_test, self.y_test
    def test(self): # 11788 all, add them to test
        print(self.X_train.shape, self.y_train.shape)
        print(self.X_test.shape, self.y_test.shape)

sunReader = sunRead(folder, train_class)
sunReader.test()
# custom
# standard 12900 + 1440

(12900, 2048) (12900,)
(1440, 2048) (1440,)


In [4]:
att = loadmat(standard_path + "att_splits.mat")
semat = att['att_original']

In [5]:
loadmat(standard_path + "att_splits.mat")['att'].shape, loadmat(standard_path + "att_splits.mat")['att_original'].shape

((102, 717), (717, 102))

In [6]:
X_train, y_train = sunReader.train_data()
X_test, y_test = sunReader.test_data()
#X_train.shape, y_train.shape, X_test.shape, y_test.shape
# hyper_parameter
pca_d = 500

exemPCA = decomposition.PCA(n_components=pca_d)
exemPCA.fit(X_train)
X_train = exemPCA.transform(X_train)
X_test = exemPCA.transform(X_test)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((12900, 500), (12900,), (1440, 500), (1440,))

In [7]:
# group up PCA projections

exem_train_group = defaultdict(list)

for c in train_class:
    exem_train_group[c] = []
        
for x, y in zip(X_train, y_train):
    exem_train_group[y].append(x)


# Average
exem_train = {}
std_train = {}
k = 0

for item in exem_train_group.items():
    y, ary = item
    exem_train[y] = np.mean(ary, axis=0) # Key Sentence
    std_train[y] = np.std(ary, axis=0)

del exem_train_group

# class to index, the same
a_c_train, a_c_test = semat[train_class], semat[test_class]
v_c_train = [exem_train[i] for i in train_class] # dict uses classes to index
sd_train = [std_train[i] for i in train_class]
sd_train = np.mean(sd_train, axis=0)

In [8]:
np.array(a_c_train).shape, np.array(v_c_train).shape, np.array(sd_train).shape

((645, 102), (645, 500), (500,))

## 第2步 
### Training

用MLP训练，


X为语义表示，是固定的a_c_train矩阵，维度为645x102转换为645x1024

$$\psi (a_c) \approx v_c$$

In [9]:
###
x, y = torch.tensor(np.array(a_c_train), dtype=torch.float), torch.tensor(np.array(v_c_train), dtype=torch.float)

In [10]:
# Torch Training

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 43, 102, 300, pca_d
# Create random Tensors to hold inputs and outputs
#x, y = torch.randn(N, D_in), torch.randn(N, D_out)
# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    #torch.nn.Dropout(p=0.5),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(3000):
    # Forward pass: compute predicted y by passing x to the model.
    for i in range(len(x)//N):
        inputs = x[i*N:i*N + N]
        labels = y[i*N:i*N + N]
        y_pred = model(inputs)
        loss = loss_fn(y_pred, labels)
        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update (which are the learnable
        # weights of the model). This is because by default, gradients are
        # accumulated in buffers( i.e, not overwritten) whenever .backward()
        # is called. Checkout docs of torch.autograd.backward for more details.
        optimizer.zero_grad()

        # Backward pass: compute gradient of the loss with respect to model
        # parameters
        loss.backward()

        # Calling the step function on an Optimizer makes an update to its
        # parameters
        optimizer.step()
    if t % 500 == 99:
        print(t, loss.item())



99 10374.3837890625
599 5953.52197265625
1099 4984.41650390625
1599 4374.96240234375
2099 3931.044921875
2599 3583.190673828125


In [11]:

test_y = model(torch.tensor(a_c_test, dtype=torch.float))
test_y.shape
v_c_test = test_y.detach().numpy()


v_c_test.shape, len(v_c_train), len(v_c_train[0])

exem_X, exem_y = [], []

for i, c in enumerate(test_class):
    exem_X.append(v_c_test[i])
    exem_y.append(c)
    
# Add this part to become GZSL
#for i, c in enumerate(train_class):
#    exem_X.append(v_c_train[i])
#    exem_y.append(c)



neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(exem_X, exem_y)
print("1NN:{} ".format(neigh.score(X_test, y_test)))

sneigh = KNeighborsClassifier(n_neighbors=1, metric='seuclidean', 
                    metric_params={'V':sd_train})
sneigh.fit(exem_X, exem_y)
print("1NNs:{}".format(sneigh.score(X_test, y_test)))

1NN:0.6125 
1NNs:0.6270833333333333


### Example Log of Training

```
99 10338.1181640625
399 6707.013671875
699 5677.1259765625
999 5096.4580078125
1299 4654.57080078125
1599 4294.96728515625
1899 4000.2236328125
2199 3748.1884765625
2499 3532.768310546875
2799 3342.576904296875

1NN:0.6090277777777777 
1NNs:0.6277777777777778
```

### Dropout + ReLU

```
99 11005.8896484375
799 6164.46337890625
1499 5249.7626953125
2199 4515.26025390625
2899 4208.16455078125
3599 4074.322509765625
4299 3824.184814453125
4999 3620.673095703125
5699 3504.84814453125
6399 3528.818115234375

1NN:0.5715277777777777 
1NNs:0.5902777777777778
```

## Original

```
regress_group = []
k = 0
for j in range(pca_d):
    X = a_c_train # 
    y = [vc[j] for vc in v_c_train]
    regressor = NuSVR(C=2.0)
    #regressor = NuSVR(C=2.0)
    regressor.fit(X, y)
    regress_group.append(regressor)

v_c_test = np.zeros((72, pca_d)) # 提前定义好exemplar矩阵, 
# 对每一个维度进行预测
for j in range(pca_d):
    v_c_test[:, j] =  regress_group[j].predict(a_c_test) 

v_c_test.shape, len(v_c_train), len(v_c_train[0])
# also add up to 717 here

exem_X, exem_y = [], []

for i, c in enumerate(test_class):
    exem_X.append(v_c_test[i])
    exem_y.append(c)
    
# Add this part to become GZSL
#for i, c in enumerate(train_class):
#    exem_X.append(v_c_train[i])
#    exem_y.append(c)



neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(exem_X, exem_y)
print("1NN:{} ".format(neigh.score(X_test, y_test)))

sneigh = KNeighborsClassifier(n_neighbors=1, metric='seuclidean', 
                    metric_params={'V':sd_train})
sneigh.fit(exem_X, exem_y)
print("1NNs:{}".format(sneigh.score(X_test, y_test)))
```

In [12]:
## Save Exemplars and visual features of test class
#save_path = "./sun2_exem/"
#np.save(save_path + "exem_test.npy", np.array(exem_X))
#np.save(save_path + "X_test.npy", X_test)
#np.savetxt(save_path + "y_test.txt", np.array(y_test, dtype=int), fmt='%s', encoding='utf-8')

In [13]:
## Proposed Split
# Their features
## original: 0.6125
## MLP variant: 0.5347222222222222
# Our features
## original: 0.5833333333333334
## MLP variant: 0.5041666666666667